In [1]:
seed=197

In [2]:
# importing required libraries
import torch.nn as nn
import torch
import torch.nn.functional as F
import math,copy,re
import warnings
import pandas as pd
import numpy as np
import random
import pickle
import torch.nn.init as init
from torchviz import make_dot
import plotly.graph_objects as go
import pickle
# import seaborn as sns
# import torchtext
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import torch.nn.functional as F
import torch.optim as optim
from IPython.display import display
import matplotlib.pyplot as plt
import os 
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

warnings.simplefilter("ignore")
print(torch.__version__)

1.13.1+cu117


In [3]:
# Check if CUDA is available
if torch.cuda.is_available():
    # CUDA is available, so PyTorch can use a GPU
    device = torch.device("cuda")
    print("PyTorch is using GPU.")
else:
    # CUDA is not available, PyTorch will use CPU
    device = torch.device("cpu")
    print("PyTorch is using CPU.")

# Create a tensor and move it to the device
tensor = torch.randn(3, 4).to(device)

# Check the device of the tensor
print("Tensor device:", tensor.device)

PyTorch is using GPU.
Tensor device: cuda:0


In [4]:
def reset_random_seeds(seed):
    os.environ['PYTHONHASHSEED']=str(seed)
#     tf.random.set_seed(seed)
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

In [5]:
# pickle_file_path = '/media/terminator/Data/NatureData/Auto Ten Minutes.pkl'
pickle_file_path = '/media/terminator/NatureData/Sixty Minutes.pkl'

In [6]:
# Load data from the pickle file
with open(pickle_file_path, "rb") as f:
    loaded_data_dict = pickle.load(f)


In [7]:
# Retrieve data and labels
X_train = loaded_data_dict["X_train"]
y_train = loaded_data_dict["y_train"]
X_test = loaded_data_dict["X_test"]
y_test = loaded_data_dict["y_test"]

In [8]:
X_train.shape,y_train.shape

(torch.Size([23328, 144, 13]), torch.Size([23328, 3]))

In [9]:
features=13

In [10]:
def reconstructData(packets):
    # Take the first packet
    first_packet = packets[0]
    
    # Concatenate the last index of every packet
    reconstructed_data = np.concatenate([packet[-1:] for packet in packets[1:]])
    
    # Combine the first packet with the concatenated data
    reconstructed_data = np.concatenate([first_packet, reconstructed_data])
    
    return reconstructed_data

In [11]:
Valid,Test = X_test[:100000],X_test[100000:]
Valid_label,Test_label = y_test[:100000],y_test[100000:]

In [12]:
num_epochs = 500
# Define batch size
batch_size = 256
# Create DataLoader for training set
train_dataset = torch.utils.data.TensorDataset(X_train.float(), y_train.float())
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
valid_dataset = torch.utils.data.TensorDataset(Valid.float(), Valid_label.float())
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_dataset = torch.utils.data.TensorDataset(Test.float(), Test_label.float())
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [13]:
print(X_train.dtype)
print(y_train.dtype)

torch.float64
torch.float64


In [14]:
X_train.shape, y_train.shape

(torch.Size([23328, 144, 13]), torch.Size([23328, 3]))

In [15]:
import torch
import torch.nn as nn

class TimeSeriesEncoder(nn.Module):
    def __init__(self, input_channels):
        super(TimeSeriesEncoder, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Conv1d(input_channels, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=1),  # Reduce sequence length to 36 / 4 = 9
            nn.Conv1d(128, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=1)  # Reduce sequence length to 9 / 2 = 4.5 (rounded to 4)
        )
        self.classifier = nn.Sequential(
            nn.Linear(64*142, 1024),
            nn.ReLU(), 
            nn.Linear(1024, 3), 
            nn.Softmax(dim=1)  
        )
        
    
    def forward(self, x):
        x = x.permute(0,2,1)
        x = self.encoder(x)
        x = torch.reshape(x,(x.shape[0],-1))
        x = self.classifier(x)
#         x = x.permute(0,2,1)
        return x

# Create the autoencoder model
input_channels = 13  # Number of input features
# encoder = TimeSeriesEncoder(input_channels).to(device)





In [16]:
train = 1
avg = 'weighted'
seq_length = 144
num_epochs = 5000
reset_random_seeds(seed)
src_vocab_size = features+5
target_vocab_size = features
num_layers = 2
latent_dim = 8
embed_dim = 128
n_heads = 8
expansion_factor = 2

Cf = TimeSeriesEncoder(input_channels).to(device)

criterion = nn.CrossEntropyLoss()
optimizer_cf = optim.Adam(Cf.parameters(), lr=0.00001)

# Define early stopping parameters
patience = 100
best_loss = 0
epochs_without_improvement = 0

if train:
    # Step 4: Training loop
    for epoch in range(num_epochs):
        running_loss = 0.0
        latents = []
        PR, PL = [], []
        train_input,train_output = [], []

        # Iterate over mini-batches
        for i, (inputs, labels) in enumerate(train_loader):
            
            probability = 0.0
            coin_toss = random.random()  

            if coin_toss < probability:
                continue
            
            # Clear gradients
            optimizer_cf.zero_grad()
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # Forward pass
            
            prediction = Cf(inputs)
            
            
            # Calculate loss
            loss = criterion(prediction, labels)
            loss = loss.float()

            # Backpropagation
            loss.backward()
            
            optimizer_cf.step()
            
            # Update running loss
            running_loss += loss.item()
            
            
            PR.append(prediction)
            PL.append(labels)
            

        # Calculate average loss per epoch
        PR = torch.cat(PR, dim=0)
        PL = torch.cat(PL, dim=0)
        _, predicted = torch.max(PR.data, 1)
        _, lbl = torch.max(PL.data, 1)
        total = lbl.size(0)
        correct = (predicted == lbl).sum().item()

        accuracy = 100 * correct / total
        print(f'Train Accuracy: {accuracy:.2f}%')
        conf_matrix = confusion_matrix(lbl.cpu().detach().numpy(),predicted.cpu().detach().numpy())
        print('Confusion Matrix:')
        print(conf_matrix)
        # Calculate average loss per epoch
        epoch_loss = running_loss / len(train_loader)
        print(f"Epoch: {epoch+1}, Train Loss: {round(epoch_loss,6)}")
        
        if epoch%1==0:
            valid_latents = []
            VR,VL = [], []
            valid_loss = 0.0
            for j, (inp, lab) in enumerate(valid_loader):
                probability = 0.0
                coin_toss = random.random()  

                if coin_toss < probability:
                    continue
                inp = inp.to(device)
                lab = lab.to(device)
                
                with torch.no_grad():
                    # Forward pass
                    outp = Cf(inp)
                    
                VR.append(outp)
                VL.append(lab)
            
            VR = torch.cat(VR, dim=0)
            VL = torch.cat(VL, dim=0)
            _, vpre = torch.max(VR.data, 1)
            _, vlbl = torch.max(VL.data, 1)
            total = vlbl.size(0)
            correct = (vpre == vlbl).sum().item()

            vaccuracy = 100 * correct / total
            print('-----------------------------------------')
            print(f'Valid Accuracy: {vaccuracy:.2f}%')
            lb = vlbl.cpu().detach().numpy()
            pre = vpre.cpu().detach().numpy()
            conf_matrix = confusion_matrix(lb,pre)
            precision = precision_score(lb, pre,average=avg)
            recall = recall_score(lb, pre,average=avg)
            f1 = f1_score(lb, pre,average=avg)
            print('Confusion Matrix:')
            print(conf_matrix)
            print()
            print("Precision:", precision)
            print("Recall:", recall)
            print("F1 Score:", f1)
            print(epochs_without_improvement, best_loss)
            print('-----------------------------------------')
            print()
            print()
            
                
                    
        # Check for early stopping and save the best weights
        th = (conf_matrix[0][0]> conf_matrix[0][1]+conf_matrix[0][2] and conf_matrix[1][1]>conf_matrix[1][0]+conf_matrix[1][2] and conf_matrix[2][2]>conf_matrix[2][0]+conf_matrix[2][1])
        if th and vaccuracy > best_loss:
            best_loss = vaccuracy
            epochs_without_improvement = 0
            # Save the model weights
            torch.save(Cf.state_dict(), 'cnn60.pt')
            print('##############################################')
            print('saved', best_loss)
            print('##############################################')
            print()
            print()
            
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= patience:
                print("Early stopping triggered. Training stopped.")
                break

# Recover the best weights
Cf.load_state_dict(torch.load('cnn60.pt'))


Train Accuracy: 47.47%
Confusion Matrix:
[[3520  827 3429]
 [1665  512 5599]
 [ 521  214 7041]]
Epoch: 1, Train Loss: 1.078274
-----------------------------------------
Valid Accuracy: 51.68%
Confusion Matrix:
[[50939     0 45544]
 [  666     0  2082]
 [   26     0   743]]

Precision: 0.9520167025483071
Recall: 0.51682
F1 Score: 0.6638764434776362
0 0
-----------------------------------------


Train Accuracy: 50.75%
Confusion Matrix:
[[4317    1 3458]
 [1836    0 5940]
 [ 254    0 7522]]
Epoch: 2, Train Loss: 1.025015
-----------------------------------------
Valid Accuracy: 52.11%
Confusion Matrix:
[[51354   150 44979]
 [  671    15  2062]
 [   26     0   743]]

Precision: 0.9545279931077726
Recall: 0.52112
F1 Score: 0.6676771278442654
1 0
-----------------------------------------


Train Accuracy: 52.88%
Confusion Matrix:
[[4315  159 3302]
 [1831  505 5440]
 [ 254    7 7515]]
Epoch: 3, Train Loss: 0.991741
-----------------------------------------
Valid Accuracy: 52.65%
Confusion Ma

Train Accuracy: 73.46%
Confusion Matrix:
[[5139 2172  465]
 [2288 4752  736]
 [ 263  268 7245]]
Epoch: 18, Train Loss: 0.816996
-----------------------------------------
Valid Accuracy: 63.66%
Confusion Matrix:
[[61259 29332  5892]
 [  783  1687   278]
 [   26    29   714]]

Precision: 0.9545450466135005
Recall: 0.6366
F1 Score: 0.7497368265948996
0 62.326
-----------------------------------------


##############################################
saved 63.66
##############################################


Train Accuracy: 73.63%
Confusion Matrix:
[[5196 2123  457]
 [2344 4736  696]
 [ 265  266 7245]]
Epoch: 19, Train Loss: 0.814426
-----------------------------------------
Valid Accuracy: 64.71%
Confusion Matrix:
[[62294 28553  5636]
 [  792  1701   255]
 [   26    29   714]]

Precision: 0.9546995998540554
Recall: 0.64709
F1 Score: 0.7575150003277835
0 63.66
-----------------------------------------


##############################################
saved 64.709
##########################

-----------------------------------------
Valid Accuracy: 66.66%
Confusion Matrix:
[[63934 27987  4562]
 [  570  1996   182]
 [   26    15   728]]

Precision: 0.9587703590569964
Recall: 0.66658
F1 Score: 0.7713609826699069
0 66.579
-----------------------------------------


##############################################
saved 66.658
##############################################


Train Accuracy: 78.67%
Confusion Matrix:
[[5306 2081  389]
 [1595 5698  483]
 [ 246  181 7349]]
Epoch: 36, Train Loss: 0.770315
-----------------------------------------
Valid Accuracy: 66.79%
Confusion Matrix:
[[64059 27898  4526]
 [  566  2000   182]
 [   26    15   728]]

Precision: 0.9588623775354503
Recall: 0.66787
F1 Score: 0.772308404470855
0 66.658
-----------------------------------------


##############################################
saved 66.787
##############################################


Train Accuracy: 78.80%
Confusion Matrix:
[[5312 2079  385]
 [1573 5721  482]
 [ 248  179 7349]]
Epoch: 

Train Accuracy: 80.29%
Confusion Matrix:
[[5369 2057  350]
 [1362 6011  403]
 [ 276  151 7349]]
Epoch: 52, Train Loss: 0.754759
-----------------------------------------
Valid Accuracy: 67.54%
Confusion Matrix:
[[64673 27553  4257]
 [  466  2137   145]
 [   29    12   728]]

Precision: 0.9605698120540743
Recall: 0.67538
F1 Score: 0.7775318656849562
0 67.46
-----------------------------------------


##############################################
saved 67.538
##############################################


Train Accuracy: 80.43%
Confusion Matrix:
[[5372 2055  349]
 [1343 6042  391]
 [ 276  151 7349]]
Epoch: 53, Train Loss: 0.754017
-----------------------------------------
Valid Accuracy: 67.57%
Confusion Matrix:
[[64694 27552  4237]
 [  466  2148   134]
 [   29    12   728]]

Precision: 0.9605883157644439
Recall: 0.6757
F1 Score: 0.7777097722698229
0 67.538
-----------------------------------------


##############################################
saved 67.57
##########################

Train Accuracy: 81.51%
Confusion Matrix:
[[5452 1999  325]
 [1206 6213  357]
 [ 292  135 7349]]
Epoch: 70, Train Loss: 0.743785
-----------------------------------------
Valid Accuracy: 67.79%
Confusion Matrix:
[[64863 27715  3905]
 [  423  2199   126]
 [   29    12   728]]

Precision: 0.9613487076424057
Recall: 0.6779
F1 Score: 0.7793033258414116
2 67.768
-----------------------------------------


##############################################
saved 67.79
##############################################


Train Accuracy: 81.52%
Confusion Matrix:
[[5453 2000  323]
 [1204 6215  357]
 [ 291  136 7349]]
Epoch: 71, Train Loss: 0.74326
-----------------------------------------
Valid Accuracy: 67.79%
Confusion Matrix:
[[64861 27727  3895]
 [  417  2205   126]
 [   29    12   728]]

Precision: 0.9614433018281631
Recall: 0.67794
F1 Score: 0.7793294438769844
0 67.79
-----------------------------------------


##############################################
saved 67.794
###########################

...

-----------------------------------------
Valid Accuracy: 91.87%
Confusion Matrix:
[[88830  5576  2077]
 [  334  2309   105]
 [   30    10   729]]

Precision: 0.9708552379266949
Recall: 0.91868
F1 Score: 0.9381417319944431
18 92.465
-----------------------------------------


Train Accuracy: 91.60%
Confusion Matrix:
[[7436  188  152]
 [ 914 6578  284]
 [ 247  175 7354]]
Epoch: 904, Train Loss: 0.634824
-----------------------------------------
Valid Accuracy: 91.78%
Confusion Matrix:
[[88742  5653  2088]
 [  333  2309   106]
 [   10    30   729]]

Precision: 0.970972408459043
Recall: 0.9178
F1 Score: 0.9376513145425244
19 92.465
-----------------------------------------


Train Accuracy: 91.60%
Confusion Matrix:
[[7436  189  151]
 [ 912 6578  286]
 [ 243  179 7354]]
Epoch: 905, Train Loss: 0.634789
-----------------------------------------
Valid Accuracy: 91.84%
Confusion Matrix:
[[88806  5589  2088]
 [  334  2309   105]
 [   30    10   729]]

Precision: 0.9708337164963975
Recall: 0.91

Train Accuracy: 91.61%
Confusion Matrix:
[[7438  186  152]
 [ 916 6578  282]
 [ 231  191 7354]]
Epoch: 924, Train Loss: 0.63469
-----------------------------------------
Valid Accuracy: 91.79%
Confusion Matrix:
[[88753  5630  2100]
 [  334  2309   105]
 [   22    18   729]]

Precision: 0.970860383060202
Recall: 0.91791
F1 Score: 0.9376759634657139
39 92.465
-----------------------------------------


Train Accuracy: 91.61%
Confusion Matrix:
[[7438  186  152]
 [ 915 6578  283]
 [ 246  176 7354]]
Epoch: 925, Train Loss: 0.634669
-----------------------------------------
Valid Accuracy: 92.00%
Confusion Matrix:
[[88963  5448  2072]
 [  348  2309    91]
 [   32     8   729]]

Precision: 0.9708362221846338
Recall: 0.92001
F1 Score: 0.9389458672036993
40 92.465
-----------------------------------------


Train Accuracy: 91.61%
Confusion Matrix:
[[7438  186  152]
 [ 916 6578  282]
 [ 237  185 7354]]
Epoch: 926, Train Loss: 0.634806
-----------------------------------------
Valid Accuracy: 92.

-----------------------------------------
Valid Accuracy: 91.91%
Confusion Matrix:
[[88868  5524  2091]
 [  334  2309   105]
 [    4    36   729]]

Precision: 0.971154314396985
Recall: 0.91906
F1 Score: 0.938494617419837
59 92.465
-----------------------------------------


Train Accuracy: 91.62%
Confusion Matrix:
[[7442  182  152]
 [ 913 6578  285]
 [ 252  170 7354]]
Epoch: 945, Train Loss: 0.634529
-----------------------------------------
Valid Accuracy: 91.84%
Confusion Matrix:
[[88801  5582  2100]
 [  333  2309   106]
 [    4    36   729]]

Precision: 0.9710968210835301
Recall: 0.91839
F1 Score: 0.9380632841801734
60 92.465
-----------------------------------------


Train Accuracy: 91.62%
Confusion Matrix:
[[7442  183  151]
 [ 912 6578  286]
 [ 249  173 7354]]
Epoch: 946, Train Loss: 0.634503
-----------------------------------------
Valid Accuracy: 91.87%
Confusion Matrix:
[[88834  5549  2100]
 [  333  2309   106]
 [    4    36   729]]

Precision: 0.971131632751541
Recall: 0.918

Train Accuracy: 91.62%
Confusion Matrix:
[[7442  182  152]
 [ 916 6578  282]
 [ 254  168 7354]]
Epoch: 965, Train Loss: 0.634733
-----------------------------------------
Valid Accuracy: 91.67%
Confusion Matrix:
[[88631  5764  2088]
 [  333  2309   106]
 [    7    33   729]]

Precision: 0.9708885251785662
Recall: 0.91669
F1 Score: 0.9369395015160535
80 92.465
-----------------------------------------


Train Accuracy: 91.62%
Confusion Matrix:
[[7442  183  151]
 [ 918 6578  280]
 [ 223  199 7354]]
Epoch: 966, Train Loss: 0.634608
-----------------------------------------
Valid Accuracy: 92.24%
Confusion Matrix:
[[89200  5180  2103]
 [  344  2309    95]
 [   39     1   729]]

Precision: 0.9710917461532411
Recall: 0.92238
F1 Score: 0.9405073970991351
81 92.465
-----------------------------------------


Train Accuracy: 91.62%
Confusion Matrix:
[[7442  183  151]
 [ 917 6578  281]
 [ 245  177 7354]]
Epoch: 967, Train Loss: 0.634604
-----------------------------------------
Valid Accuracy: 9

<All keys matched successfully>

In [20]:
Cf.load_state_dict(torch.load('cnn60.pt'))

<All keys matched successfully>

In [21]:
valid_latents = []
VR,VL = [], []
valid_loss = 0.0
for j, (inp, lab) in enumerate(train_loader):
    probability = 0.0
    coin_toss = random.random()  

    if coin_toss < probability:
        continue
    inp = inp.to(device)
    lab = lab.to(device)

    with torch.no_grad():
        # Forward pass
        outp = Cf(inp)

    VR.append(outp)
    VL.append(lab)

VR = torch.cat(VR, dim=0)
VL = torch.cat(VL, dim=0)
_, vpre = torch.max(VR.data, 1)
_, vlbl = torch.max(VL.data, 1)
total = vlbl.size(0)
correct = (vpre == vlbl).sum().item()

vaccuracy = 100 * correct / total
print('-----------------------------------------')
print(f'Train Accuracy: {vaccuracy:.2f}%')
lb = vlbl.cpu().detach().numpy()
pre = vpre.cpu().detach().numpy()
conf_matrix = confusion_matrix(lb,pre)
precision = precision_score(lb, pre,average=avg)
recall = recall_score(lb, pre,average=avg)
f1 = f1_score(lb, pre,average=avg)
print('Confusion Matrix:')
print(conf_matrix)
print()
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print(epochs_without_improvement, best_loss)
print('-----------------------------------------')
print()
print()

valid_latents = []
VR,VL = [], []
valid_loss = 0.0
for j, (inp, lab) in enumerate(valid_loader):
    probability = 0.0
    coin_toss = random.random()  

    if coin_toss < probability:
        continue
    inp = inp.to(device)
    lab = lab.to(device)

    with torch.no_grad():
        # Forward pass
        outp = Cf(inp)

    VR.append(outp)
    VL.append(lab)

VR = torch.cat(VR, dim=0)
VL = torch.cat(VL, dim=0)
_, vpre = torch.max(VR.data, 1)
_, vlbl = torch.max(VL.data, 1)
total = vlbl.size(0)
correct = (vpre == vlbl).sum().item()

vaccuracy = 100 * correct / total
print('-----------------------------------------')
print(f'Valid Accuracy: {vaccuracy:.2f}%')
lb = vlbl.cpu().detach().numpy()
pre = vpre.cpu().detach().numpy()
conf_matrix = confusion_matrix(lb,pre)
precision = precision_score(lb, pre,average=avg)
recall = recall_score(lb, pre,average=avg)
f1 = f1_score(lb, pre,average=avg)
print('Confusion Matrix:')
print(conf_matrix)
print()
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print(epochs_without_improvement, best_loss)
print('-----------------------------------------')
print()
print()

valid_latents = []
VR,VL = [], []
valid_loss = 0.0
for j, (inp, lab) in enumerate(test_loader):
    probability = 0.0
    coin_toss = random.random()  

    if coin_toss < probability:
        continue
    inp = inp.to(device)
    lab = lab.to(device)

    with torch.no_grad():
        # Forward pass
        outp = Cf(inp)

    VR.append(outp)
    VL.append(lab)

VR = torch.cat(VR, dim=0)
VL = torch.cat(VL, dim=0)
_, vpre = torch.max(VR.data, 1)
_, vlbl = torch.max(VL.data, 1)
total = vlbl.size(0)
correct = (vpre == vlbl).sum().item()

vaccuracy = 100 * correct / total
print('-----------------------------------------')
print(f'Test Accuracy: {vaccuracy:.2f}%')
lb = vlbl.cpu().detach().numpy()
pre = vpre.cpu().detach().numpy()
conf_matrix = confusion_matrix(lb,pre)
precision = precision_score(lb, pre,average=avg)
recall = recall_score(lb, pre,average=avg)
f1 = f1_score(lb, pre,average=avg)
print('Confusion Matrix:')
print(conf_matrix)
print()
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print(epochs_without_improvement, best_loss)
print('-----------------------------------------')
print()
print()

-----------------------------------------
Train Accuracy: 91.58%
Confusion Matrix:
[[7432  191  153]
 [ 904 6578  294]
 [ 413    9 7354]]

Precision: 0.9208869784354304
Recall: 0.9158093278463649
F1 Score: 0.9158807273160834
100 92.465
-----------------------------------------


-----------------------------------------
Valid Accuracy: 92.47%
Confusion Matrix:
[[89427  4947  2109]
 [  336  2309   103]
 [   39     1   729]]

Precision: 0.9714506332355827
Recall: 0.92465
F1 Score: 0.9420484880246268
100 92.465
-----------------------------------------


-----------------------------------------
Test Accuracy: 94.02%
Confusion Matrix:
[[150484   6454   2661]
 [   536   2863    112]
 [    54      1    917]]

Precision: 0.9769263505458952
Recall: 0.9401640643093088
F1 Score: 0.954174115431449
100 92.465
-----------------------------------------




In [19]:
 # Iterate over mini-batches
latents = []
train_label = []
for i, (inputs, labels, lbl) in enumerate(train_loader):

    inputs = inputs.to(device)
    lbl = lbl.to(device)
    
    with torch.no_grad():

        # Forward pass
        latent = Encoder(inputs)


    latents.append(latent)
    train_label.append(lbl)


ValueError: not enough values to unpack (expected 3, got 2)

In [ ]:
latents = torch.cat(latents,dim=0)

In [ ]:
train_label = torch.cat(train_label,dim=0)

In [ ]:
latents = latents.cpu().detach().numpy()

In [ ]:
rs = reconstructData(latents)

In [ ]:
torch.argmax(train_label,dim=1)

In [ ]:
labl = torch.argmax(train_label,dim=1).cpu().detach().numpy()

In [ ]:
rs.shape

In [ ]:
real_data_reshaped = np.reshape(latents, (latents.shape[0], -1))


# Perform tsne on real and fake data
perplexity = min(30, real_data_reshaped.shape[0] - 1)
tsne = TSNE(n_components=2, perplexity=perplexity, random_state=seed)
real_tsne = tsne.fit_transform(real_data_reshaped)
label_names = ['No-Movement', 'Small Movement',  'Large Movement']
# Plot the t-SNE visualization with labeled data
plt.figure(figsize=(20, 15))
for label in np.unique(labl):
    indices = np.where(labl == label)
    plt.scatter(real_tsne[indices, 0], real_tsne[indices, 1], label=label_names[int(label)], edgecolors='black')
    
plt.legend()
plt.show()

In [ ]:
-----------------------------------------
Train Accuracy: 91.58%
Confusion Matrix:
[[7432  191  153]
 [ 904 6578  294]
 [ 413    9 7354]]

Precision: 0.9208869784354304
Recall: 0.9158093278463649
F1 Score: 0.9158807273160834
100 92.465
-----------------------------------------


-----------------------------------------
Valid Accuracy: 92.47%
Confusion Matrix:
[[89427  4947  2109]
 [  336  2309   103]
 [   39     1   729]]

Precision: 0.9714506332355827
Recall: 0.92465
F1 Score: 0.9420484880246268
100 92.465
-----------------------------------------


-----------------------------------------
Test Accuracy: 94.02%
Confusion Matrix:
[[150484   6454   2661]
 [   536   2863    112]
 [    54      1    917]]

Precision: 0.9769263505458952
Recall: 0.9401640643093088
F1 Score: 0.954174115431449
100 92.465
-----------------------------------------

In [ ]:
-----------------------------------------
Train Accuracy: 88.61%
Confusion Matrix:
[[6682  954  140]
 [ 945 6710  121]
 [ 435   62 7279]]

Precision: 0.8875690656187784
Recall: 0.8861025377229081
F1 Score: 0.8866658095578258
100 80.479
-----------------------------------------


-----------------------------------------
Valid Accuracy: 80.48%
Confusion Matrix:
[[77819 17188  1475]
 [  763  1940    46]
 [   33    16   720]]

Precision: 0.9603073569758481
Recall: 0.80479
F1 Score: 0.8661478300003829
100 80.479
-----------------------------------------


-----------------------------------------
Test Accuracy: 84.33%
Confusion Matrix:
[[135089  22386   2124]
 [  1087   2374     50]
 [    47     23    902]]

Precision: 0.9683679846076022
Recall: 0.8432673906948964
F1 Score: 0.8945902175201007
100 80.479
-----------------------------------------